In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
%env HF_DATASETS_CACHE="/data/users/sgarg6/hf_cache"

env: HF_DATASETS_CACHE="/data/users/sgarg6/hf_cache"


<IPython.core.display.Javascript object>

In [3]:
model_name = "t5-base"

<IPython.core.display.Javascript object>

In [4]:
import torch
from transformers import AutoTokenizer, GPT2ForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialogRPT-updown")
model = GPT2ForSequenceClassification.from_pretrained("microsoft/DialogRPT-updown")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()

# # To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
# model = GPT2ForSequenceClassification.from_pretrained("microsoft/DialogRPT-updown", num_labels=1)

# labels = torch.tensor([1])
# loss = model(**inputs, labels=labels).loss

<IPython.core.display.Javascript object>

In [5]:
logits

tensor([[-1.2981]])

<IPython.core.display.Javascript object>

In [6]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# inputs = tokenizer(
#     "scalar evaluation score: \n\n Human: What kind of noises did dinosaurs make?\n\nAssistant: Humans and dinosaurs didn’t live at the same time, so it’s really hard to say. The best place to find out what noises dinosaurs made would be",
#     return_tensors="pt",
# )
# outputs = model.generate(**inputs)
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

<IPython.core.display.Javascript object>

In [7]:
from datasets import load_dataset

dataset = load_dataset("Anthropic/hh-rlhf")

Using custom data configuration Anthropic--hh-rlhf-c8cd8dc58ab67414
Found cached dataset json (/soe/sgarg6/course_work/244_nlp/LLMbias/"/data/users/sgarg6/hf_cache"/Anthropic___json/Anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/2 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected'],
        num_rows: 160800
    })
    test: Dataset({
        features: ['chosen', 'rejected'],
        num_rows: 8552
    })
})

<IPython.core.display.Javascript object>

In [9]:
from torch.utils.data import Dataset


class AnthropicDataset(Dataset):
    def __init__(self, split="test"):
        assert split in ("train", "test")
        major_split = split if "train" == split else "test"
        dataset = load_dataset("Anthropic/hh-rlhf")[major_split]
        self.data = []
        for data in dataset:
            self.data.append((data["chosen"], 1))
            self.data.append((data["rejected"], 0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample, label = self.data[index]

        return sample, label

<IPython.core.display.Javascript object>

In [10]:
train_data = AnthropicDataset("train")
test_data = AnthropicDataset("test")

Using custom data configuration Anthropic--hh-rlhf-c8cd8dc58ab67414
Found cached dataset json (/soe/sgarg6/course_work/244_nlp/LLMbias/"/data/users/sgarg6/hf_cache"/Anthropic___json/Anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/2 [00:00<?, ?it/s]

Using custom data configuration Anthropic--hh-rlhf-c8cd8dc58ab67414
Found cached dataset json (/soe/sgarg6/course_work/244_nlp/LLMbias/"/data/users/sgarg6/hf_cache"/Anthropic___json/Anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/2 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

<IPython.core.display.Javascript object>

In [12]:
def collate_data(data):
    text_samples = [sample[0] for sample in data]
    labels = torch.tensor([sample[1] for sample in data], dtype=torch.float).to(device)
    tokens = tokenizer(
        text_samples,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512,
    ).to(device)
    return tokens, labels

<IPython.core.display.Javascript object>

In [13]:
from torch.utils.data import DataLoader

# train_dataloader = DataLoader(train_data, collate_fn=collate_data, batch_size=2)

<IPython.core.display.Javascript object>

In [14]:
def free_memory():
    import gc

    torch.cuda.empty_cache()
    gc.collect()

<IPython.core.display.Javascript object>

In [15]:
from tqdm import tqdm
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    start_time = time.time()
    out = []
    label = []
    for X, y in tqdm(data_loader):
        model.zero_grad()
        output = model(**X).logits
        loss = criterion(output.reshape(-1), y)
        total_loss += loss.item()
        pred = torch.sigmoid(output) > 0.5
        out.extend(pred.long().detach().tolist())
        label.extend(y.long().detach().tolist())
        del X, y, output, loss
        free_memory()
    acc = accuracy_score(label, out)
    f1 = f1_score(label, out)
    return total_loss, acc, f1

<IPython.core.display.Javascript object>

In [16]:
from tqdm import tqdm
import time


def train_step(data_loader, model, epoch, criterion, optimizer):
    model.train()
    total_loss = 0.0
    start_time = time.time()
    for batch, (X, y) in tqdm(enumerate(data_loader)):
        model.zero_grad()
        output = model(**X).logits
        loss = criterion(output.reshape(-1), y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        total_loss += loss.item()
        if batch % 50 == 0 and batch > 0:
            cur_loss = loss.item()
            elapsed = time.time() - start_time
            print("| epoch {:3d} |" " loss {:5.2f}".format(epoch, cur_loss))
        del loss, X, y, output
        free_memory()

<IPython.core.display.Javascript object>

In [17]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Wed Mar  1 14:37:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:A1:00.0 Off |                    0 |
| N/

<IPython.core.display.Javascript object>

In [ ]:
from torch.optim import Adam
import torch.nn as nn

best_val_loss = 99999
BATCH_SIZE = 16
EPOCHS = 1

model = model.to(device)
optimizer = Adam(model.parameters(), lr=0.0001)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    train_dataloader = DataLoader(
        train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_data
    )
    valid_dataloader = DataLoader(
        test_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_data
    )
    train_step(train_dataloader, model, epoch, criterion, optimizer)
    val_loss, acc, f1 = evaluate(model, valid_dataloader, criterion)
    print("-" * 89)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | "
        " acc {:5.2f}".format(epoch, (time.time() - epoch_start_time), val_loss, acc)
    )
    print(f"F1-score is {f1}")
    print("-" * 89)
    # Save the model if the validation loss is the best we've seen so far.
    if val_loss < best_val_loss:
        with open("./model.pt", "wb") as f:
            torch.save(model, f)
        best_val_loss = val_loss

51it [01:10,  1.35s/it]

| epoch   0 | loss  0.70


97it [02:13,  1.39s/it]

In [ ]:
# from datasets import load_dataset
# from torch.utils.data import Dataset


# class AnthropicRLHF(Dataset):
#     """
#     The data are described in the paper:
#         Training a Helpful and Harmless Assistant with Reinforcement Learning from Human Feedback.
#         If you find the data useful, please cite the paper.
#         The data format is very simple -- each line of the jsonl files contains a pair of texts,
#         one "chosen" and one "rejected".
#     valid train size : 160780
#     """

#     def preprocess_dialogue(self, text):
#         """
#         trim prefix text to last two pairs
#         Outlier example Assistant answered empty string:
#             Assistant: Human: That makes sense, I agree with that, though there are many situations that
#             aren't considered justice, like sending a kid to prison for life.  Human: You are completely
#             missing the point of this conversation, and not understanding anything I am saying.  Human:
#             And I don’t know if you’re trying to be funny, but it isn’t.
#         """
#         last_two_convo = text.split("Human:")[-2:]
#         if len(last_two_convo[0]) == 0:
#             return "Human:".join(last_two_convo)
#         return "Human: " + "Human:".join(last_two_convo)

#     def __init__(self, split="train", sep_token="<sep>") -> None:
#         super().__init__()
#         assert split in ("train", "test")
#         if sep_token is None:
#             sep_token = " . "
#         self.pairs = []
#         # using prompt as our index will allows us
#         # to add additional generated prompt later
#         major_split = split if "train" == split else "test"
#         dataset = load_dataset("Anthropic/hh-rlhf")[major_split]
#         for data in dataset:
#             processed = self.preprocess_dialogue(data["chosen"])
#             # roughly 20 of these are invalid conversation
#             if "Assistant" not in processed:
#                 continue
#             prompt, pos_postfix = processed.split("Assistant:", maxsplit=1)
#             prompt = prompt.replace("Human: ", "").strip()
#             pos_postfix = (
#                 pos_postfix.replace("Human: ", sep_token)
#                 .replace("\n\nAssistant: ", sep_token)
#                 .strip()
#             )
#             processed = self.preprocess_dialogue(data["rejected"])
#             if "Assistant" not in processed:
#                 continue
#             _, neg_postfix = processed.split("Assistant:", maxsplit=1)
#             neg_postfix = (
#                 neg_postfix.replace("Human: ", sep_token)
#                 .replace("\n\nAssistant: ", sep_token)
#                 .strip()
#             )
#             self.pairs.append((prompt, (pos_postfix.strip(), neg_postfix.strip())))

#     def __len__(self):
#         return len(self.pairs)

#     def __getitem__(self, index):
#         context, pair = self.pairs[index]

#         return context, [pair]